In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
ord=pd.read_csv("Order Report.csv")
invoice=pd.read_csv("Invoice.csv")
cc=pd.read_csv("Courier Company - Rates.csv")
pincode=pd.read_csv("pincodes.csv")
sku=pd.read_csv("SKU Master.csv")

In [3]:
print(f"Order head \n {ord.head()}")
print(f"Invoice \n {invoice.head()}")
print(f"Courier Company \n {cc.head()}")
print(f"Pincode \n {pincode.head()}")
print(f"SKU \n {sku.head()}")

Order head 
    ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN
Invoice 
         AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0  1091117222124  2001806232            1.30             121003   
1  1091117222194  2001806273            1.00             121003   
2  1091117222931  2001806408            2.50             121003   
3  1091117223244  2001806458            1.00             121003   
4  1091117229345  2001807012            0.15             121003   

   Customer Pincode Zone Type of Shipment  Billing Amount (Rs.)  
0            507101    d  Forward charges                 135.0  
1            486886    d  Forward charge

In [4]:
# Check for missing values
print("\nMissing values in Website Order Report:")
print(ord.isnull().sum())
print("\nMissing values in SKU Master:")
print(sku.isnull().sum())
print("\nMissing values in Pincode Mapping:")
print(pincode.isnull().sum())
print("\nMissing values in Courier Invoice:")
print(invoice.isnull().sum())
print("\nMissing values in courier company rates:")
print(cc.isnull().sum())


Missing values in Website Order Report:
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

Missing values in SKU Master:
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

Missing values in Pincode Mapping:
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64

Missing values in Courier Invoice:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing values in courier company rates:
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed        

In [5]:
# Remove unnamed columns from the Website Order Report DataFrame
ord = ord.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

# Remove unnamed columns from the SKU Master DataFrame
sku = sku.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

# Remove unnamed columns from the Pincode Mapping DataFrame
pincode = pincode.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

In [6]:
# Merge the Order Report and SKU Master based on SKU
merged_data = pd.merge(ord, sku, on='SKU')
print(merged_data.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001827036  8904223819093        1.0         150
2     2001827036  8904223819109        1.0         100
3     2001827036  8904223818430        1.0         165
4     2001827036  8904223819277        1.0         350


In [7]:
# Rename the "ExternOrderNo" column to "Order ID" in the merged_data DataFrame
merged_data = merged_data.rename(columns={'ExternOrderNo': 'Order ID'})

In [8]:
#merge the courier invoice and pincode mapping dataset
pincode_courier = pincode.drop_duplicates(subset=['Customer Pincode'])
invoice_courier = invoice[['Order ID', 'Customer Pincode','Type of Shipment']]
pincodes = invoice_courier.merge(pincode_courier,on='Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


In [9]:
merged2 = merged_data.merge(pincodes,on='Order ID')
merged2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e


In [10]:
#Calculating weight in Kilograms
merged2['Weight (kgs)'] = merged2['Weight (g)']/1000
merged2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Weight (kgs)
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350


In [11]:
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5

merged2['Weight Slab (KG)'] = merged2['Weight (kgs)'].apply(weight_slab)
invoice['Weight Slab Charged by Courier Company'] = invoice['Charged Weight'].apply(weight_slab)

In [12]:
# Rename the columns names
invoice = invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per ABC'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

In [13]:
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per ABC  Weight (kgs)  \
0  Forward charges             121003                        e         0.127   
1  Forward charges             121003                        e         0.150   
2  Forward charges             121003                        e         0.100   
3  Forward charges             121003                        e         0.165   
4  Forward charges             121003                        e         0.350   

   Weight Slab As Per ABC  
0                     0.5  
1                     0.

In [14]:
# Calculate the charges expected
expected_charges = []
for _,row in merged2.iterrows():
    fwd_category = 'fwd_' + row ['Delivery Zone As Per ABC']
    fwd_fixed = cc.at[0, fwd_category + '_fixed']
    fwd_additional = cc.at[0,fwd_category + '_additional']
    rto_category = 'rto_' + row ['Delivery Zone As Per ABC']
    rto_fixed = cc.at[0, rto_category + '_fixed']
    rto_additional = cc.at[0,rto_category + '_additional']

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        expected_charges.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        expected_charges.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        expected_charges.append(0)

merged2['Expected Charges as per ABC'] = expected_charges
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per ABC  Weight (kgs)  \
0  Forward charges             121003                        e         0.127   
1  Forward charges             121003                        e         0.150   
2  Forward charges             121003                        e         0.100   
3  Forward charges             121003                        e         0.165   
4  Forward charges             121003                        e         0.350   

   Weight Slab As Per ABC  Expected Charges as per ABC  
0                     0

In [15]:
#Merge Invoice data with merged2 data
merged_invoice=merged2.merge(invoice, on='Order ID')
print(merged_invoice.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode_x  \
0  2001827036  8904223818706        1.0         127              173213   
1  2001827036  8904223819093        1.0         150              173213   
2  2001827036  8904223819109        1.0         100              173213   
3  2001827036  8904223818430        1.0         165              173213   
4  2001827036  8904223819277        1.0         350              173213   

  Type of Shipment_x  Warehouse Pincode_x Delivery Zone As Per ABC  \
0    Forward charges               121003                        e   
1    Forward charges               121003                        e   
2    Forward charges               121003                        e   
3    Forward charges               121003                        e   
4    Forward charges               121003                        e   

   Weight (kgs)  Weight Slab As Per ABC  Expected Charges as per ABC  \
0         0.127                     0.5                 

In [16]:
#Difference in charges and expected charges
merged_invoice['Difference (Rs.)'] = merged_invoice['Billing Amount (Rs.)'] - merged_invoice['Expected Charges as per ABC']
df_new=merged_invoice[['Order ID','Difference (Rs.)','Expected Charges as per ABC']]
print (df_new.head())

     Order ID  Difference (Rs.)  Expected Charges as per ABC
0  2001827036              61.3                         56.6
1  2001827036              61.3                         56.6
2  2001827036              61.3                         56.6
3  2001827036              61.3                         56.6
4  2001827036              61.3                         56.6


In [17]:
# Calculate the total orders in each category
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

# Calculate the total amount in each category
amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charges as per ABC'].sum()

# Create a new DataFrame for the summary
summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)

print(df_summary)


                                         Description  Count  Amount (Rs.)
0  Total Orders where ABC has been correctly charged     12         507.6
1        Total Orders where ABC has been overcharged    382       33750.5
2       Total Orders where ABC has been undercharged      7        -165.2


In [18]:
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            textinfo='label+percent',
                            hole=0.6))
fig.update_layout(title='Proportion')

fig.show()